In [ ]:
from scipy.interpolate import interp1d
from scipy.ndimage import rotate
from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import trimesh
import pyembree
import random
from NRSS.writer import write_materials, write_hdf5, write_config, write_slurm
from NRSS.checkH5 import checkH5

from Morphology import Morphology
from Fibril import Fibril
import sys
import pathlib
sys.path.append('/home/devon/Documents/Github/NRSS/')

%load_ext autoreload
%autoreload 2
%matplotlib inline
#https://stackoverflow.com/questions/53999426/how-to-parameterize-a-curved-cylinder

In [ ]:
# Declare model box size in nm (x,y,z)
x_dim_nm  = 1024
y_dim_nm  = 1024
z_dim_nm  = 128
pitch_nm = 2 # Dimension of voxel in nm

# Initialize morphology
morphology = Morphology(x_dim_nm, y_dim_nm, z_dim_nm, pitch_nm, 2)
morphology.set_model_parameters(radius_nm_avg = 15.,
                                radius_nm_std = 1.0,
                                max_num_fibrils = 150,
                                fibril_length_range_nm = [100, 400])

morphology.fill_model()

In [ ]:
morphology.voxelize_model()

In [ ]:
phys_size = 2.0
write_hdf5([[morphology.mat1_Vfrac, morphology.mat1_S, morphology.mat1_theta, morphology.mat1_psi], 
            [morphology.mat2_Vfrac, morphology.mat2_S, morphology.mat2_theta, morphology.mat2_psi]],
            phys_size, 'Fibril.hdf5')

In [ ]:
energies = np.round(np.arange(270., 340., 1),1)
energies

In [ ]:
write_config(list(energies), [0.0, 1.0, 360.0], CaseType=0, MorphologyType=0)

In [ ]:
write_slurm('Fibril.hdf5')

In [ ]:
checkH5('Fibril.hdf5', z_slice=40, plotstyle='dark')

In [ ]:
import subprocess
subprocess.run(['CyRSoXS','Fibril.hdf5'])

In [ ]:
import h5py
basePath = pathlib.Path('.').absolute()
h5path = pathlib.Path(basePath,'HDF5')
h5list = list(sorted(h5path.glob('*h5')))

In [ ]:
def print_key(f, key):
    try:
        keys2 = f[key].keys()
        for key2 in keys2:
            new_key = key + '/' + key2
            print_key(f, new_key)
    except AttributeError:
        print(key)

with h5py.File(h5list[0],'r') as f:
    for key in f.keys():
        print_key(f, key)

In [ ]:
from matplotlib import cm
from matplotlib.colors import LogNorm

In [ ]:
import sys
sys.path.append('/home/devon/Documents/Github/PyHyperScattering/src/')

In [ ]:
from PyHyperScattering.load import cyrsoxsLoader
from PyHyperScattering.integrate import WPIntegrator

In [ ]:
load = cyrsoxsLoader()
integ = WPIntegrator(force_np_backend=True) # avoiding gpu backend for this tutorial

In [ ]:
raw = load.loadDirectory(basePath)

In [ ]:
remeshed = integ.integrateImageStack(raw)

In [ ]:
c = cm.jet(np.linspace(0,1,len(remeshed)))

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,3),dpi=140,constrained_layout=True)
raw.sel(energy=275).plot(norm=LogNorm(1e-2,1e7),cmap='terrain',ax=ax[0],add_colorbar=False)
raw.sel(energy=284).plot(norm=LogNorm(1e-2,1e7),cmap='terrain',ax=ax[1],add_colorbar=False)
raw.sel(energy=300).plot(norm=LogNorm(1e-2,1e7),cmap='terrain',ax=ax[2])

[{axes.set_xlim(-0.4,0.4),axes.set_ylim(-0.4,0.4)} for axes in ax]
plt.show()

In [ ]:
# calculate the anisotropy metric
A = remeshed.rsoxs.AR(chi_width=20)

In [ ]:
A.plot(x='q',cmap='bwr',vmin=-1,vmax=1)
plt.xlim(1e-3)
plt.xscale('log')
plt.show()

In [ ]:
A.sel(energy=299,method='nearest').plot()
plt.show()

In [ ]:
horz = remeshed.rsoxs.slice_chi(0, chi_width=20)
vert = remeshed.rsoxs.slice_chi(90, chi_width=20)

In [ ]:
vert.plot(x='q',cmap='terrain',norm=LogNorm(1e-5,1e7), xscale='linear')
plt.xlim(left=0.01, right=1)
plt.show()

In [ ]:
vert.plot(x='q',cmap='terrain',norm=LogNorm(1e-5,1e7), xscale='linear')
plt.xlim(left=0.01, right=1)
plt.show()

In [ ]:
vert.sel(energy=339).plot(yscale='log',xscale='linear')
plt.xlim(left=0.01, right=1)
plt.ylim(bottom=1e-5, top=1e2)
plt.show()

In [ ]:
horz.sel(energy=275).plot(xscale='log',yscale='log',label='275 eV Horizontal',color='tab:blue')
vert.sel(energy=275).plot(xscale='log',yscale='log',label='275 eV Vertical',linestyle='--',color='tab:blue')
horz.sel(energy=284,method='nearest').plot(xscale='log',yscale='log',label='284 eV Horizontal',color='tab:orange')
vert.sel(energy=284).plot(xscale='log',yscale='log',label='275 eV Vertical',linestyle='--',color='tab:orange')
horz.sel(energy=287,method='nearest').plot(xscale='log',yscale='log',label='287 eV Horizontal',color='tab:green')
vert.sel(energy=287).plot(xscale='log',yscale='log',label='275 eV Vertical',linestyle='--',color='tab:green')
plt.legend()
plt.xlabel(r'q [nm$^{-1}$]')
plt.ylabel('I(q)')
plt.show()

In [ ]:
scene = morphology.get_scene(show_bounding_box=True)
scene.show()

In [ ]:
scene = morphology.get_scene(show_bounding_box=True, show_voxelized=True)
# scene.show()

In [ ]:
indices = np.array([[0,0,0]],dtype=int)

fibrils = morphology.fibrils
for fibril in fibrils:
    voxel_mesh = fibril.voxel_mesh
    indices = np.append(indices, np.array(voxel_mesh.vertices, dtype=int),axis=0)

indices  = [index for index in indices if index[0] < morphology.x_dim and index[1] < morphology.y_dim and index[2] < morphology.z_dim]
    
voxel_box = np.zeros((morphology.x_dim, morphology.y_dim, morphology.z_dim))
for index in indices:
    voxel_box[tuple(index)] = 1

In [ ]:
fibril.voxel_mesh.vertices

In [ ]:
plt.close()
fig = plt.figure(dpi=300)
def updatefig(i):
    fig.clear()
    plt.imshow(morphology.orientation_box[:,:,i,:])
    plt.draw()
anim = animation.FuncAnimation(fig, updatefig, 64)
anim.save("test.mp4", fps=16)
plt.close()

In [ ]:
orientation = np.random.random(3)
orientation /= np.linalg.norm(orientation)
orientation

In [ ]:
np.linalg.norm(orientation)

In [ ]:
fibril_volume = 0
fibril_vox_volume = 0
for fibril in morphology.fibrils:
    fibril_volume     += fibril.volume
    fibril_vox_volume += fibril.voxel_volume
print(fibril_volume)
print(f'Fibril Volume    : {fibril_volume:.2f}')
print(f'Fibril Vox Volume: {fibril_vox_volume:.2f}')
print(f'Box Volume: {morphology.box_volume}')
mpc = fibril_volume / morphology.box_volume * 100
vpc = fibril_vox_volume / morphology.box_volume * 100
print(f'Mesh Percent Crystallinity: {mpc:.2f}')
print(f'Voxel Percent Crystallinity: {vpc:.2f}')

- Animation for morphology
- List of model parameters

In [ ]:
ma = morphology.mesh_list[0]
va = ma.voxelized(pitch=pitch)

mb = morphology.mesh_list[1]
vb = mb.voxelized(pitch=pitch)

In [ ]:
vmesh_list = []
for mesh in tqdm(morphology.mesh_list):
    v = mesh.voxelized(pitch=1)
    vmesh = v.fill().as_boxes()
    vmesh_list.append(vmesh)

vscene = trimesh.Scene(vmesh_list)
vscene.show()

In [ ]:
vscene = trimesh.Scene(vmesh_list)
vscene.show()

In [ ]:
indices = np.array([[0,0,0]],dtype=int)
for vmesh in vmesh_list:
    indices = np.append(indices, np.array(vmesh.vertices, dtype=int),axis=0)
# np.shape(indices)
indices  = [index for index in indices if index[0] < morphology.x_dim and index[1] < morphology.y_dim and index[2] < morphology.z_dim]
    
voxel_box = np.zeros((morphology.x_dim, morphology.y_dim, morphology.z_dim))
for index in indices:
    voxel_box[tuple(index)] = 1

In [ ]:
vmesh_list[0].vertices

In [ ]:
np.any(vmesh_list[0].vertices > 500)

In [ ]:
plt.figure()
plt.imshow(voxel_box[:,:,9])
# np.where(voxel_box == 1)

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
ax.voxels(voxel_box)
ax.set_aspect('equal')
plt.show()

In [ ]:
# print(vmesh_list)
# vmesh_list.append(morphology.bounding_path)
vscene = trimesh.Scene(vmesh_list)
vscene.show()

In [ ]:
plt.imshow(v.matrix[:,10])
plt.show()

In [ ]:
v.translation

In [ ]:
vmesh_list.pop()

### What's next?
###### - Get the angles sorted
###### - Functionality to check average angle
###### - Check degree of crystallinity
###### - Voxelize mesh
###### - Angles 

In [ ]:
full_cyl_vox = full_cyl_mesh.voxelized(pitch=2)
full_cyl_vox.fill()
full_cyl_vox_mesh  = full_cyl_vox.as_boxes()
print(full_cyl_vox.volume)
print(full_cyl_mesh.volume)

In [ ]:
full_cyl_vox_mesh.show()

In [ ]:
broken = trimesh.repair.broken_faces(full_cyl_vox_mesh, color=[255, 0, 0, 255])
print(len(broken))
print(full_cyl_vox_mesh.is_watertight)
full_cyl_vox_mesh.show(smooth=False)

In [ ]:
core_cyl_vox = core_cyl_mesh.voxelized(pitch=2)
core_cyl_vox.fill()
core_cyl_vox_mesh  = core_cyl_vox.as_boxes()
print(core_cyl_vox.volume)
print(core_cyl_mesh.volume)

In [ ]:
shell_cyl_vox = shell_cyl_mesh.voxelized(pitch=2)
shell_cyl_vox.fill()
shell_cyl_vox_mesh = shell_cyl_vox.as_boxes()
print(shell_cyl_vox.volume)
print(shell_cyl_mesh.volume)

In [ ]:
within_mesh(full_cyl_vox_mesh, core_cyl_vox_mesh)

In [ ]:
shell_cyl_mesh.is_watertight

In [ ]:
v1_mesh.visual.face_colors[:] = [255, 0, 0, 0]
trans_matrix = trimesh.transformations.translation_matrix([64, 0, 0])
v2_mesh.apply_transform(trans_matrix)
voxel_scene = trimesh.Scene([v3_mesh, m3])
voxel_scene.show()

In [ ]:
v

In [ ]:
radius = 10
height = 40
pitch  = 0.2
mesh = trimesh.primitives.Cylinder(radius=radius, height=height, use_embree=True)
mesh_vol = mesh.volume
print(f'Mesh Volume: {mesh_vol:.2f}')

voxel_grid = mesh.voxelized(pitch=pitch)
voxel_mesh = voxel_grid.fill().as_boxes()
voxel_mesh_vol = voxel_mesh.volume
print(f'Voxel Mesh Volume: {voxel_mesh_vol:.2f}')

vol_diff = np.abs(voxel_mesh_vol - mesh_vol)/mesh_vol * 100
print(f'Volume Difference: {vol_diff:.2f}%')

In [ ]:
import numpy as np

In [ ]:
mu, sigma = 0, 1.0
samples   = 100000
gnoise = np.random.normal(mu, sigma, samples)
enoise = np.random.uniform(0.0, np.pi, samples)
abs(mu - np.mean(gnoise))

theta = (90 + gnoise)/180 * np.pi
phi    = enoise

In [ ]:
vec2 = np.sin(theta)*np.cos(phi)
vec1 = np.sin(theta)*np.sin(phi)
vec0 = np.cos(theta)
plt.figure()
count, bins, ignored = plt.hist(vec2, 30, density=True)
plt.title('vec2')
plt.figure()
count, bins, ignored = plt.hist(vec1, 30, density=True)
plt.title('vec1')
plt.figure()
count, bins, ignored = plt.hist(vec0, 30, density=True)
plt.title('vec0')
plt.show()